# Självstudieövningar – Vecka 4

De här övningarna är till för att du ska repetera och befästa det vi har gått igenom under veckan samt introducera några nya begrepp på ett tryggt sätt.

Det innebär att vissa av uppgifterna kan du behöva söka efter information själv hur du löser. Här kan du använda Google eller AI om du vill men Python-dokumentationen är din bästa vän när du lär dig.

Länk: https://docs.python.org/3.13/

Läs först igenom vad som ska göras i respektive uppgift och fyll sedan i kodblocket nedanför, eller gör uppgiften i en separat fil.

**Detta lämnas inte in för rättning.** Arbeta i din egen takt, testa dig fram, använd dokumentationen när du kör fast och ta med eventuella frågor till nästa lektionstillfälle.

## (Kör först) Skapa veckans dataset

In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)  # reproducible
n = 300
cities = np.array(["Göteborg","Malmö","Stockholm","Uppsala"])
genders = np.array(["F","M"])

age = rng.integers(18, 65, size=n)
gender = rng.choice(genders, size=n)
city = rng.choice(cities, size=n, p=[0.3,0.2,0.4,0.1])
steps = rng.normal(8000, 2000, size=n).clip(1000, 20000).astype(int)
sleep_hours = rng.normal(7.0, 1.2, size=n).clip(3.5, 11.0)
screen_time = rng.normal(4.0, 1.5, size=n).clip(0.5, 12.0)
calories = rng.normal(2200, 450, size=n).clip(1000, 4000)

# relationer: fler steg → lägre skärmtid, högre kalorier
screen_time = np.clip(screen_time - (steps - 8000)/8000 * 0.6, 0.5, 12.0)
calories    = np.clip(calories    + (steps - 8000)/8000 * 120, 1000, 4000)

df = pd.DataFrame({
    "age": age,
    "gender": gender,
    "city": city,
    "steps": steps,
    "sleep_hours": np.round(sleep_hours,2),
    "screen_time": np.round(screen_time,2),
    "calories": np.round(calories,0).astype(int)
})

# ca 2% saknade värden (NaN)
mask = rng.random(size=df.shape) < 0.02
for (r, c), m in np.ndenumerate(mask):
    if m:
        df.iat[r, c] = np.nan

df.to_csv("week4_data.csv", index=False)
print("Skapade week4_data.csv med", len(df), "rader.")

### Om datasettet (week4_data.csv)

Det här är ett **syntetiskt** dataset från en fiktiv hälso-/livsstilsapp. Varje rad är en person/dag.

| Kolumn | Typ | Beskrivning | Ungefärligt intervall |
|---|---|---|---|
| `age` | int | Ålder | 18–65 |
| `gender` | kategori | Kön (F/M) | – |
| `city` | kategori | Stad | Göteborg, Malmö, Stockholm, Uppsala |
| `steps` | int | Antal steg/dygn | ~1 000–20 000 |
| `sleep_hours` | float | Sömn/dygn | ~3.5–11 |
| `screen_time` | float | Skärmtid/dygn | ~0.5–12 |
| `calories` | int | Energiförbrukning | ~1 000–4 000 |

**Noteringar:** Fast seed för reproducerbarhet, ~2% NaN, svag relation: fler `steps` → lägre `screen_time` men högre `calories` i snitt.

### Snabb kontroll
- Läs in `week4_data.csv` i `df`.
- Visa `df.head()` för att se de första raderna.
- Visa datatyper med `df.info()`.
- Räkna saknade värden per kolumn med en metod i Pandas.
- Skriv en kort kommentar (i en kodkommentar) om vilken kolumn som verkar ha flest NaN.

In [ ]:
import pandas as pd
df = pd.read_csv("week4_data.csv")
display(df.head())
df.info()

na_counts = df.isna().sum()
print("\nSaknade värden per kolumn:")
print(na_counts)

display(df.describe())

## Block A – NumPy

### Uppgift 1 – Array-grunder
Skapa en följd av heltal och gör enkla operationer med **vektoriserad** aritmetik.
- Skapa en 1D-array med heltal 0..19.
- Skapa en ny array med kvadraterna av dessa tal (utan loop).
- Plocka ut de tal som är delbara med 3 genom ett boolean-filter.

In [ ]:
arr = np.arange(20)
squares = arr * arr
div3 = arr[arr % 3 == 0]
print(arr); print(squares); print(div3)

### Uppgift 2 – Slicing & reshaping
Träna indexering i 2D.
- Skapa en array med tal 1..24.
- Forma om till en 4×6-matris.
- Plocka ut kolumnerna 2–4 (inklusive) och raderna 2–3 (inklusive) via slicing.

In [ ]:
A = np.arange(1, 25).reshape(4, 6)
sub = A[1:3, 1:4]
print(A); print(sub)

### Uppgift 3 – Broadcasting
Använd broadcasting för att kombinera en matris och en vektor.
- Skapa en 3×3-matris med ettor.
- Skapa en vektor `[10, 20, 30]`.
- Lägg till vektorn **radvis** till matrisen och skriv ut resultatet.

In [ ]:
A = np.ones((3,3))
v = np.array([10,20,30])
print(A + v)

### Uppgift 4 – Felsökning (listor vs arrayer)
Koden nedan försöker summera elementvis men använder listor. Kör och rätta.
```python
import numpy as np
a = [1,2,3]
b = [4,5,6]
print(a + b)      # blir listkonkatenering
print(a * b)      # TypeError
```
**Mål:** gör om till arrayer så att addition/multiplikation blir elementvis.

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])
print(a + b)
print(a * b)

### Uppgift 5 – Simulera & jämför tärningar
Simulera slump och jämför fördelningar.
- Skapa två separata simuleringar med `np.random.integers()`. Simulera 1000 kast där utfallet ligger mellan 1 och 6.
- Räkna hur många ettor, tvåor, ... sexor i respektive simulering.
- Jämför simuleringarna (t.ex. med absolut differens mellan frekvenserna) och skriv ut en liten tabell.
*(tips: en funktion i NumPy: `np.bincount` kan räkna förekomster per heltal)*

In [ ]:
rng = np.random.default_rng(0)
s1 = rng.integers(1,7, size=1000)
s2 = rng.integers(1,7, size=1000)
f1 = np.bincount(s1-1, minlength=6)
f2 = np.bincount(s2-1, minlength=6)
diff = np.abs(f1 - f2)
pd.DataFrame({"utfall": np.arange(1,7), "sim1": f1, "sim2": f2, "abs_diff": diff})

## Block B – Pandas (inkl. rensa data)

### Uppgift 6 – Skapa och inspektera DataFrame
- Skapa en DataFrame **manuellt** från en dictionary (3–4 rader), t.ex. namn/ålder/stad.
- Visa de första raderna.
- Visa datatyper och en numerisk sammanfattning.

In [ ]:
df_demo = pd.DataFrame({"name":["Ada","Bob","Clara","Dan"],"age":[30,25,40,22],"city":["Göteborg","Malmö","Stockholm","Uppsala"]})
display(df_demo.head())
df_demo.info()
df_demo.describe(include="all")

### Uppgift 7 – Välja & filtrera
- Läs `week4_data.csv`.
- Skapa en vy som bara innehåller kolumnerna `age`, `steps`, `city`.
- Filtrera fram rader där `steps` är större än 10 000 **och** `city` är Göteborg.
- Visa de 5 första raderna av resultatet.

In [ ]:
df = pd.read_csv("week4_data.csv")
demo_df = df[["age","steps","city"]].copy()
demo_df[(demo_df["steps"]>10000) & (demo_df["city"]=="Göteborg")].head()

### Uppgift 8 – Sortera & skapa kolumner
- Sortera data fallande på `steps` (störst först).
- Skapa en ny kolumn `steps_k` som visar steg i tusental (avrunda till 1 decimal).
- Visa de 5 första raderna efter sorteringen.

In [ ]:
df2 = df.sort_values("steps", ascending=False).copy()
df2["steps_k"] = (df2["steps"]/1000).round(1)
df2.head()

### Uppgift 9 – Saknade värden
- Räkna antalet saknade värden per kolumn.
- Fyll saknade i `sleep_hours` med **medianen**.
- Fyll saknade i `steps` med **medelvärdet**.
- Kontrollera igen att saknade värden minskat för dessa kolumner.

In [ ]:
df_clean = df.copy()
print("Före:\n", df.isna().sum())
df_clean["sleep_hours"] = df_clean["sleep_hours"].fillna(df["sleep_hours"].median())
df_clean["steps"] = df_clean["steps"].fillna(df_clean["steps"].mean())
print("Efter:\n", df_clean.isna().sum())

### Uppgift 10 – Dubbletter
- Räkna hur många rader som är dubbletter.
- Ta bort dubblettrader.
- Visa radantal före och efter borttagningen.

In [ ]:
before = len(df_clean)
dups = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
after = len(df_clean)
before, dups, after

### Uppgift 11 – Typer & konvertering
- Konvertera `steps` till numeriskt på ett **säkert** sätt så att text som inte går att tolka blir saknad (NaN).
- Fyll dessa saknade värden i `steps` med kolumnens medelvärde.
- Jämför `dtypes` före och efter.
*(tips: titta på en Pandas-funktion som börjar `pd.to_num...`)*

In [ ]:
num = pd.to_numeric(df_clean["steps"], errors="coerce")
df_clean["steps"] = num.fillna(num.mean())
df_clean["steps"].dtype, df_clean["steps"].isna().sum()

### Uppgift 12 – Rengöringspipeline
Gör en liten pipeline som kopplar ihop allt:
- Hantera saknade värden (se uppgift 9/11).
- Skapa en ny kolumn `active_ratio = steps / (screen_time * 1000)`.
- Filtrera bort orimliga rader (t.ex. `sleep_hours < 4` eller `> 12`).
- Gruppera per `city` och skriv ut medelvärden för `steps` och `active_ratio`.

In [ ]:
df_clean = df.copy()
df_clean["sleep_hours"] = df_clean["sleep_hours"].fillna(df_clean["sleep_hours"].median())
df_clean["steps"] = pd.to_numeric(df_clean["steps"], errors="coerce").fillna(df_clean["steps"].mean())
df_clean["screen_time"] = pd.to_numeric(df_clean["screen_time"], errors="coerce")
df_clean["active_ratio"] = df_clean["steps"] / (df_clean["screen_time"]*1000)
df_clean = df_clean[(df_clean["sleep_hours"]>=4) & (df_clean["sleep_hours"]<=12)]
res = (df_clean.groupby("city")
        .agg(steps_mean=("steps","mean"),
             active_mean=("active_ratio","mean"))
        .sort_values("steps_mean", ascending=False))
res

## Block C – NumPy + Pandas (deskriptiv statistik)

### Uppgift 13 – Centrala mått med NumPy
- Ta `steps` som **NumPy-array** med `df['steps'].to_numpy()`.
- Ignorera saknade värden innan du räknar.
- Räkna ut **medelvärde**, **median** och **standardavvikelse**.  
*(tips: `np.mean`, `np.median`, `np.std`)*

In [ ]:
arr = pd.read_csv("week4_data.csv")["steps"].to_numpy(dtype="float64")
arr = arr[~np.isnan(arr)]
np.mean(arr), np.median(arr), np.std(arr, ddof=0)

### Uppgift 14 – Fem-tals-sammanfattning
För `sleep_hours`:
- Räkna fram min, Q1 (25%), median, Q3 (75%), max.
- Ignorera saknade värden först.  
*(tips: det finns en NumPy-funktion som heter `np.percentile`)*

In [ ]:
x = df["sleep_hours"].to_numpy(dtype="float64")
x = x[~np.isnan(x)]
q1 = np.percentile(x,25)
med = np.median(x)
q3 = np.percentile(x,75)
{"min": float(np.min(x)), "Q1": float(q1), "med": float(med), "Q3": float(q3), "max": float(np.max(x))}

### Uppgift 15 – IQR och outliers
- Räkna IQR = Q3 − Q1 för `screen_time`.
- Sätt gränserna `low = Q1 - 1.5*IQR` och `high = Q3 + 1.5*IQR`.
- Skapa en DataFrame `outliers_df` med de rader som ligger utanför intervallet och skriv hur många de är.

In [ ]:
st = df["screen_time"].to_numpy(dtype="float64")
st = st[~np.isnan(st)]
q1,q3 = np.percentile(st,[25,75])
iqr = q3-q1
low, high = q1-1.5*iqr, q3+1.5*iqr
outliers_df = df[(df["screen_time"]<low) | (df["screen_time"]>high)]
len(outliers_df), outliers_df.head()

### Uppgift 16 – Felsökning (typkonvertering)
Koden försöker räkna medelvärde på blandade typer. Kör och rätta.
```python
import numpy as np
vals = ["3", "5", "7", None]
print(np.mean(vals))  # TypeError
```
**Mål:** gör om värdena till tal och ignorera ogiltiga poster.

In [ ]:
vals = ["3", "5", "7", None]
nums = []
for v in vals:
    try:
        nums.append(float(v))
    except (TypeError, ValueError):
        pass
np.mean(nums)

### Uppgift 17 – Sammanfattning per stad
- Skapa en tabell/gruppsammanställning med `city` som index och kolumnerna:
  - `steps_mean` (medel), `sleep_median` (median), `screen_std` (standardavvikelse).
- Sortera resultatet fallande på `steps_mean`.
*(tips: `groupby(...).agg({...})` är en väg)*

In [ ]:
res = (df.groupby("city")
         .agg(steps_mean=("steps","mean"),
              sleep_median=("sleep_hours","median"),
              screen_std=("screen_time","std"))
         .sort_values("steps_mean", ascending=False))
res

## Block D – Matplotlib

### Uppgift 18 – Stapeldiagram från frekvenser
- Räkna frekvens av `city` med `value_counts()`.
- Rita ett **stapeldiagram**.
- Lägg till titel, axelrubriker och rutnät för att göra grafen lätt att läsa.
*(tips: du kan antingen använda `DataFrame.plot(...)` eller `matplotlib.pyplot`)*

In [ ]:
import matplotlib.pyplot as plt
ax = df["city"].value_counts().plot(kind="bar")
ax.set_title("Frekvens per stad")
ax.set_xlabel("Stad")
ax.set_ylabel("Antal")
ax.grid(True, axis="y")
plt.tight_layout()
plt.show()

### Uppgift 19 – Fördelning (histogram)
- Rita ett **histogram** av `steps` (välj ett rimligt antal bins, t.ex. 20).
- Lägg till titel/axlar och ett rutnät.
- Skriv en kort kommentar i kod (t.ex. om fördelningen ser skev ut).

In [ ]:
ax = df["steps"].dropna().plot(kind="hist", bins=20)
ax.set_title("Fördelning av steps")
ax.set_xlabel("Steps")
ax.set_ylabel("Frekvens")
ax.grid(True, axis="y")
plt.tight_layout()
plt.show()

### Uppgift 20 – Samband (scatter)
- Skapa en **scatterplot** med `steps` på x-axeln och `calories` på y-axeln.
- Lägg till titel, axelrubriker och rutnät.
- Skriv en kort kommentar om du tycker sambandet verkar positivt/negativt/svagt.

In [ ]:
ax = df_clean.plot.scatter(x="steps", y="calories")
ax.set_title("Steps vs Calories")
ax.grid(True)
plt.tight_layout()
plt.show()

### Uppgift 21 – Felsökning (plot)
Koden försöker rita ett histogram på en icke-existerande kolumn. Kör och rätta.
```python
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("week4_data.csv")
df["step"].plot(kind="hist", bins=10)
plt.show()
```
**Mål:** använd korrekt kolumn och visa grafen.

In [ ]:
import pandas as pd, matplotlib.pyplot as plt
df = pd.read_csv("week4_data.csv")
ax = df["steps"].plot(kind="hist", bins=10)
ax.set_title("Histogram av steps")
plt.show()

### Uppgift 22 – Mini-dashboard
Skapa tre figurer:
1. Histogram av `sleep_hours`,
2. Stapeldiagram med frekvenser för `gender`,
3. Scatter `steps` vs `screen_time`.

Lägg på titlar/axlar/rutnät och spara **en** figur till fil (valfritt filnamn).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 6))
axes[0,0].hist(df_clean["sleep_hours"], bins=20)
axes[0,0].set_title("Sleep hours")
axes[0,0].set_xlabel("hours")
axes[0,0].grid(True, axis="y")

axes[0,1].bar(df_clean["gender"].value_counts().index, df_clean["gender"].value_counts().values)
axes[0,1].set_title("Gender frequences")
axes[0,1].set_xlabel("gender")
axes[0,1].set_ylabel("antal")
axes[0,1].grid(True, axis="y")

axes[1,0].scatter(df_clean["steps"], df_clean["screen_time"])
axes[1,0].set_title("Steps vs Screen time")
axes[1,0].set_xlabel("steps")
axes[1,0].set_ylabel("screen time")
axes[1,0].grid(True)

axes[1,1].axis("off")

plt.tight_layout()
plt.savefig("dashboard.png", dpi=300)